<a href="https://colab.research.google.com/github/Juanvargs/SYS/blob/main/Copia1_de_Detectordeaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit librosa joblib pyngrok yt-dlp gdown

In [ ]:
%%writefile app.py
import streamlit as st
import librosa
import librosa.display
import numpy as np
import joblib
import os
import yt_dlp
import gdown  # Para descargar desde Google Drive
import matplotlib.pyplot as plt

#https://drive.google.com/file/d/12IPvTtEBg0joAP4MseWcR8oiQLr0raT6/view?hechoporjuanpablovargasusp=sharing

st.set_page_config(page_title="Detector de Género Musical")
st.title(" Detector de Género Musical: Reguetón vs Salsa")

# 🔗 ID del archivo en Google Drive
FILE_ID = "12IPvTtEBg0joAP4MseWcR8oiQLr0raT6"
modelo_path = "/content/reggaeton_vs_salsa.pkl"

#  Descargar modelo si no existe localmente
if not os.path.exists(modelo_path):
    try:
        st.info("📥 Descargando modelo desde Google Drive...")
        gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", modelo_path, quiet=False)
    except Exception as e:
        st.error(f"Error descargando el modelo: {e}")
        st.stop()

#  Cargar modelo
try:
    modelo_dict = joblib.load(modelo_path)
    modelo = modelo_dict['modelo'] if isinstance(modelo_dict, dict) else modelo_dict
    st.success("Modelo cargado correctamente.")
except Exception as e:
    st.error(f"Error cargando el modelo: {e}")
    st.stop()

# Descargar audio desde YouTube como MP3
def download_ytvid_as_mp3(video_url, name):
    output = f"/content/{name}"
    options = {
        'format': 'bestaudio/best',
        'outtmpl': output,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            final_path = f"{output}.mp3"
            return final_path if os.path.exists(final_path) else None
        except Exception as e:
            st.error(f"Error al descargar audio: {e}")
            return None

# 🎛️ Extraer características (FFT normalizada)
def extraer_fft(file_path):
    try:
        y, sr = librosa.load(file_path, sr=22050)
        fft = np.abs(np.fft.fft(y))
        fft = fft[:len(fft)//2]

        # Normalizar como en el entrenamiento
        if np.max(fft) != 0:
            fft = fft / np.max(fft)

        if len(fft) < 120001:
            fft = np.pad(fft, (0, 120001 - len(fft)), mode='constant')
        else:
            fft = fft[:120001]

        return fft.reshape(1, -1), y, sr  # ← también retornamos la señal y sr
    except Exception as e:
        st.error(f"Error al procesar el audio: {e}")
        return None, None, None

# Diccionario de clases
CLASES = {
    1.0: "Reguetón",
    2.0: "Salsa"
}

# Entrada del usuario
url = st.text_input(" Ingresa el enlace de la canción (YouTube):")

if url:
    with st.spinner("descargando audio..."):
        mp3_file = download_ytvid_as_mp3(url, "audio")
        if not mp3_file:
            st.error("❌ El archivo MP3 no fue encontrado después de la descarga.")
            st.stop()


    with st.spinner("🔍 Clasificando género musical..."):
        try:
            pred = modelo.predict(features)[0]
            genero = CLASES.get(pred, "Desconocido")
            st.success(f"🎵 Género detectado: **{genero}**")
        except Exception as e:
            st.error(f"❌ Error al clasificar: {e}")


In [ ]:
#### ACA DIGALE A CHAT GPT QUE LE ENSEÑE COMO CREAR EL AUTOKHEN DE NGROK Y QUE LUEGO POR MEDIO DE SE LE DE EL CODIGO PARA PUBLICAR LA URL DE LA APP###